In [2]:
import pandas as pd
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [3]:
df1 = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet')
df2 = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet')

In [4]:
df1.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00


## Q1

In [5]:
len(df1.columns)

19

In [6]:
df1.columns

Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag',
       'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra',
       'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
       'total_amount', 'congestion_surcharge', 'airport_fee'],
      dtype='object')

## Q2

In [7]:
df1['duration'] = (df1['tpep_dropoff_datetime']- df1['tpep_pickup_datetime'])/pd.Timedelta(minutes=1)

In [8]:
df1['duration'].std()

42.594351241920904

## Q3

In [9]:
#number of rows in the original dataframe
df1.shape[0]

3066766

In [10]:
#after filtering
df1_filtered = df1[(df1['duration'] >= 1) & (df1['duration'] <= 60)]

In [11]:
100*df1_filtered.shape[0]/df1.shape[0]

98.1220282212598

## Q4

In [12]:
dv = DictVectorizer()
df1_X = df1_filtered[['PULocationID', 'DOLocationID']].astype(str)
df1_X_dicts = df1_X.to_dict(orient='records')
X_train = dv.fit_transform(df1_X_dicts)

In [13]:
len(dv.feature_names_)

515

## Q5

In [14]:
lr = LinearRegression()

In [15]:
y_train = df1_filtered['duration']

In [16]:
lr.fit(X_train, y_train)

LinearRegression()

In [17]:
y_pred_train = lr.predict(X_train)

In [18]:
RMSE_train = (mean_squared_error(y_train, y_pred_train))**0.5

In [19]:
RMSE_train

7.6492610279057605



## Q6

In [20]:
df2['duration'] = (df2['tpep_dropoff_datetime']- df2['tpep_pickup_datetime'])/pd.Timedelta(minutes=1)

In [21]:
df2 = df2[(df2['duration'] >= 1) & (df2['duration'] <= 60)]

In [22]:
df2_X_val = df2[['PULocationID', 'DOLocationID']].astype(str)
df2_X_val_dicts = df2_X_val.to_dict(orient='records')

In [23]:
X_val = dv.transform(df2_X_val_dicts)

In [24]:
y_pred_val = lr.predict(X_val)

In [25]:
RMSE_val = (mean_squared_error(df2['duration'], y_pred_val))**0.5

In [26]:
RMSE_val

7.81183265470218